# Export Outputs to a JSON file 

In this guide, we will demonstrate how to serialize the check's and suite's output as a JSON file.
This enables export the results as a serializeable object that can later be shared or reviewed in other Python scopes.

## Load Data 

In [6]:
from deepchecks.datasets.classification import iris

train_dataset, test_dataset = iris.load_data()
model = iris.load_fitted_model()

## Run a Check

In [10]:
from deepchecks.checks import ModelErrorAnalysis

res = ModelErrorAnalysis().run(train_dataset, test_dataset, model)

In [11]:
res

Model Error Analysis Find features that best split the data into segments of high and low model error. Additional Outputs 
 The following graphs show the distribution of error for top features that are most useful for distinguishing
 high error samples from low error samples. Top features are calculated using `feature_importances_`.

## Serialize the results to a JSON format 

Serialization of the output to a JSON format is possible using the `to_json` function. This function takes the check outputs and serializes it to a JSON string.

In [15]:
serialized_output = res.to_json()

serialized_output

'{"name": "Model Error Analysis", "parameters": {}, "value": {"scorer_name": "Accuracy", "feature_segments": {"petal length (cm)": {"segment1": {"score": {"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/reduce": [{"py/type": "numpy.dtype"}, {"py/tuple": ["f8", false, true]}, {"py/tuple": [3, "<", null, null, null, -1, -1, 0]}]}, {"py/b64": "cT0K16Nw7T8="}]}]}, "n_samples": 25, "frac_samples": 0.6578947368421053}, "segment2": {"score": {"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/id": 7}, {"py/b64": "AAAAAAAA8D8="}]}]}, "n_samples": 13, "frac_samples": 0.34210526315789475}}, "petal width (cm)": {"segment1": {"score": {"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/id": 7}, {"py/b64": "cT0K16Nw7T8="}]}]}, "n_samples": 25, "frac_samples": 0.6578947368421053}, "segment2": {"score": {"py/reduce": [{"py/function": "numpy.core.multiarray.scalar"}, {"py/tuple": [{"py/id": 7}, {"py/b64": "

## Display a serialized output

The serialized JSON can be used to reproduce the run in other Python scores, by using the `display_from_json` function

In [16]:
from deepchecks import CheckResult

CheckResult.display_from_json(serialized_output)

Model Error Analysis

Find features that best split the data into segments of high and low model error.

Additional Outputs

The following graphs show the distribution of error for top features that are most useful for distinguishing
 high error samples from low error samples. Top features are calculated using `feature_importances_`.

## Serializing and Deseralizing a Suite's Output

### Running a Suite

In [17]:
from deepchecks.suites import full_suite

suite = full_suite()

In [18]:
suite_result = suite.run(train_dataset=train_dataset, test_dataset=test_dataset, model=model)

Full Suite:   0%|          | 0/36 [00:00<?, ? Check/s]

### Serialize to JSON 

In [20]:
serialized_suite = suite_result.to_json()
serialized_suite

'["{\\"name\\": \\"Model Info\\", \\"parameters\\": {}, \\"value\\": {\\"type\\": \\"RandomForestClassifier\\", \\"params\\": {\\"bootstrap\\": true, \\"ccp_alpha\\": 0.0, \\"class_weight\\": null, \\"criterion\\": \\"gini\\", \\"max_depth\\": null, \\"max_features\\": \\"auto\\", \\"max_leaf_nodes\\": null, \\"max_samples\\": null, \\"min_impurity_decrease\\": 0.0, \\"min_samples_leaf\\": 1, \\"min_samples_split\\": 2, \\"min_weight_fraction_leaf\\": 0.0, \\"n_estimators\\": 100, \\"n_jobs\\": null, \\"oob_score\\": false, \\"random_state\\": null, \\"verbose\\": 0, \\"warm_start\\": false}}, \\"display\\": [{\\"py/tuple\\": [\\"header\\", \\"<h4>Model Info</h4>\\"]}, {\\"py/tuple\\": [\\"docs_header\\", \\"<p>Summarize given model parameters.</p>\\"]}, {\\"py/tuple\\": [\\"outputs_header\\", \\"<h5>Additional Outputs</h5>\\"]}, {\\"py/tuple\\": [\\"html\\", \\"Model Type: RandomForestClassifier\\"]}, {\\"py/tuple\\": [\\"dataframe\\", \\"[{\\\\\\"Parameter\\\\\\":\\\\\\"bootstrap\\\\

### Display the Suite from a serialized JSON 

In [21]:
from deepchecks import SuiteResult

SuiteResult.display_from_json(serialized_suite)

Model Info

Summarize given model parameters.

Additional Outputs

Model Type: RandomForestClassifier

,Parameter,Value,Default
0,bootstrap,True,True
1,ccp_alpha,0.00,0.00
2,class_weight,None,None
3,criterion,gini,gini
4,max_depth,None,None
5,max_features,auto,auto
6,max_leaf_nodes,None,None
7,max_samples,None,None
8,min_impurity_decrease,0.00,0.00
9,min_samples_leaf,1,1


...

Label Ambiguity - Test Dataset

Find samples with multiple labels.

Conditions Summary

,Status,Condition,More Info
0,✓,Ambiguous sample ratio is not greater than 0%,


Additional Outputs